In [1]:
import requests
from bs4 import BeautifulSoup
from google.colab import files
import csv
import os
import re
import pandas as pd

In [2]:
DOMAIN = 'https://www.nfl.com/stats/player-stats/category'

# Lista para armazenar as URLs encontradas
URLs = [f'{DOMAIN}/field-goals/2024/post/all/kickingfgmade/desc',
        f'{DOMAIN}/field-goals/2024/REG/all/kickingfgmade/DESC?aftercursor=AAAAGQAAABlANQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNU0lzSWpNeU1EQTBNalUxTFRVME1EWXRNVFkwTWkwek56UTFMVGMxWTJVMVpHVmpNRFF5TWlJc0lqSXdNalFpWFgwPQ==',
        f'{DOMAIN}/kickoffs/2024/reg/all/kickofftotal/desc',
        f'{DOMAIN}/kickoffs/2024/REG/all/kickofftotal/DESC?aftercursor=AAAAGQAAABlATwAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJNaUlzSWpNeU1EQTBNalUxTFRVME1EWXRNVFkwTWkwek56UTFMVGMxWTJVMVpHVmpNRFF5TWlJc0lqSXdNalFpWFgwPQ==',
        f'{DOMAIN}/kickoffs/2024/REG/all/kickofftotal/DESC?aftercursor=AAAAMgAAADJAGAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STJJaXdpTXpJd01EVXdOREV0TlRRME9TMDRPVEl3TFdNeU1UVXRZemMxTjJabVlqQm1NVGxsSWl3aU1qQXlOQ0pkZlE9PQ==',
        f'{DOMAIN}/kickoff-returns/2024/reg/all/kickreturnsaverageyards/desc',
        f'{DOMAIN}/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAGQAAABlAPzMzMzMzMzFleUp6WldGeVkyaEJablJsY2lJNld5SXpNUzR5SWl3aU16SXdNRFJrTkRrdE5UUXpOeTAyTmpVekxXUm1ObU10Tm1VNVpqVTNNalV6WmpNM0lpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAMgAAADJAO9R64UeuFDFleUp6WldGeVkyaEJablJsY2lJNld5SXlOeTQ0TXlJc0lqTXlNREEwWWpRMUxUUmxOakV0TkRZeU5TMHhPVFpsTFdZM1pUWTBaRE0wWkRrNE5DSXNJakl3TWpRaVhYMD0=',
        f'{DOMAIN}/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAASwAAAEtAOlR64UeuFDFleUp6WldGeVkyaEJablJsY2lJNld5SXlOaTR6TXlJc0lqTXlNREEwTWpReExUUmlNak10TWpVeU1pMWxZbVUyTFRSaU5qRXpaRE01TkdKbE1DSXNJakl3TWpRaVhYMD0=',
        f'{DOMAIN}/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAZAAAAGRAOIAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlOQzQxSWl3aU16SXdNRFE0TkRFdE5USXpOaTB6T0RjeExUSm1NakV0TVRKbE5EYzNNVFl4T1RRMElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/kickoff-returns/2024/REG/all/kickreturnsaverageyards/DESC?aftercursor=AAAAfQAAAHhANQAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXlNUzR3SWl3aU16SXdNRFUwTkRrdE5UQTJNaTA0TlRVM0xUUTJNbU10TmpSak5ESmpOalUzWTJJeklpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/punts/2024/reg/all/puntingaverageyards/desc',
        f'{DOMAIN}/punts/2024/REG/all/puntingaverageyards/DESC?aftercursor=AAAAGQAAABlAR0AAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STBOaTQxSWl3aU16SXdNRFUzTlRJdE5EazJOQzA1T0RBNUxUZzBabVl0TXprelpERmhOR1EwTkRreElpd2lNakF5TkNKZGZRPT0=',
        f'{DOMAIN}/punt-returns/2024/reg/all/puntreturnsaverageyards/desc',
        f'{DOMAIN}/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAAGQAAABlAJKPXCj1wpDFleUp6WldGeVkyaEJablJsY2lJNld5SXhNQzR6TWlJc0lqTXlNREEwTVRVMUxUVXpOek10T0RBd05DMDFabVl6TFRVeVpqRmpNbUkwWm1Jd1lpSXNJakl3TWpRaVhYMD0=',
        f'{DOMAIN}/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAAMgAAADJAIAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5STRMakFpTENJek1qQXdOR0UxTlMwMU16Z3hMVEkzTXpZdE5qazBNQzB5WmpWak1tRXdPREl6WWpjaUxDSXlNREkwSWwxOQ==',
        f'{DOMAIN}/punt-returns/2024/REG/all/puntreturnsaverageyards/DESC?aftercursor=AAAASwAAAEYAAAAAAAAAADFleUp6WldGeVkyaEJablJsY2lJNld5SXdMakFpTENJek1qQXdOVE0wWXkwME1UVXdMVEF3TnpjdE16RTFaaTAwTjJObVpqSTROemRpTWpNaUxDSXlNREkwSWwxOQ=='
]

HEADERS = {'Uer-Agent': 'Mozilla/5.0'}

In [3]:
def get_page_content(url, headers):
  response = requests.get(url, headers=headers)
  soup = BeautifulSoup(response.text, 'html.parser')
  return soup

In [4]:
# Lista para armazenar as URLs dos jogadores e das imagens corrigidas
url_players = []

# Loop sobre todas as URLs principais
for url in URLs:
    # Obter o conteúdo da página para a URL atual
    soup = get_page_content(url, HEADERS)

    # Localizar a tabela de jogadores na página
    table = soup.find('table', {'class': 'd3-o-table d3-o-table--detailed d3-o-player-stats--detailed d3-o-table--sortable'})

    # Iterar sobre todas as linhas que contêm os links para os jogadores
    for row in table.find_all('a', {'class': 'd3-o-player-fullname nfl-o-cta--link'}):
        # Adicionar o link de cada jogador à lista
        url_players.append(row['href'])

In [5]:
url_players = pd.unique(pd.Series(url_players))

In [6]:
# Função para escrever dados no arquivo CSV
def write_to_csv(filename, data, headers):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in data:
            writer.writerow(row)

# Listas para armazenar os dados dos QBs e RBs
qb_data = []

# Loop sobre todas as URLs dos jogadores
for url in url_players:
    # Coletar informações do time do jogador em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats/career', HEADERS)

    # Extrair a posição do jogador
    position_tag = soup.find('span', {'class': 'nfl-c-player-header__position'})
    player_position = position_tag.text.strip() if position_tag else 'Unknown'

    # Extrair o time do jogador
    team_tag = soup.find_all('td')
    player_team = team_tag[1].text.strip() if team_tag else 'Unknown'

    # Dados do time em 2024
    soup = get_page_content(f'https://www.nfl.com{url}stats', HEADERS)

    table = soup.find('table')
    rows = table.find_all('tr')

    if player_position == "K":
        for row in rows:
            # Extrair as células da linha
            cells = row.find_all('td')
            if len(cells) > 0:
                game_data = [
                    player_team,           # Nome do time
                    cells[0].text.strip(), # WK
                    cells[8].text.strip(), # XP ATT
                    cells[9].text.strip(), # XP
                ]
                qb_data.append(game_data)

# Salvar os dados em arquivos CSV
write_to_csv('Extra_Points.csv', qb_data, [
    'team', 'week', 'attempts', 'extra_points'
])